<a href="https://colab.research.google.com/github/venu72561-chinnam/AI-ML-project/blob/main/Mistral_MPT_multi_model_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers accelerate sentencepiece

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [ ]:
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
def chat_mistral(prompt):
    formatted_prompt = f"[INST] {prompt} [/INST]"
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=400)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
print(chat_mistral("what is AI"))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] what is AI [/INST] AI stands for Artificial Intelligence. It refers to the development of computer systems or machines that can perform tasks that typically require human intelligence, such as visual perception, speech recognition, decision-making, and language translation. AI systems use various techniques, including machine learning, deep learning, and natural language processing, to analyze data and make decisions based on that data. The ultimate goal of AI is to create intelligent machines that can work and learn like humans, and potentially even surpass human intelligence in certain areas.


In [ ]:
import os

In [ ]:
offload_dir = "/content/mpt_offload"
os.makedirs(offload_dir, exist_ok=True)

In [ ]:
model_id = "mosaicml/mpt-7b-instruct"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,      # use bfloat16 for Colab GPU
    device_map="auto",               # auto spread between GPU and CPU
    offload_folder=offload_dir,      # <— key fix
)

tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin.index.json: 0.00B [00:00, ?B/s]

...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.36G [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
def chat_mpt(prompt):
    input_text = f"### Instruction:\n{prompt}\n### Response:\n"
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=400)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
print(chat_mpt("Explain supervised vs unsupervised learning in simple terms."))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [ ]:
import gradio as gr

def gradio_chat(prompt, model_choice):
    if model_choice == "Mistral":
        return chat_mistral(prompt)
    else:
        return chat_mpt(prompt)

gr.Interface(
    fn=gradio_chat,
    inputs=["text", gr.Radio(["Mistral", "MPT"], label="Choose Model")],
    outputs="text",
    title="Chatbot"
).launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fd63b8ba30dfbcdb33.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
